In [ ]:
import mlflow

from generalist.tools import eda_table_tool, write_code_tool, execute_code_tool
from generalist.models.core import MLFlowLLMWrapper
from generalist.tools import code
from clog import logging

mlflow.langchain.autolog()                                                     # this is needed to register traces within the experiment
experiment_name = f"tool_test"
mlflow.set_experiment(experiment_name)
logging.getLogger().setLevel(logging.INFO)
# MONKEY PATCH THE LLM CALLS TO HAVE THE LOGGING IN STDOUT
code.llm = MLFlowLLMWrapper(code.llm)

In [ ]:
tools = [eda_table_tool, write_code_tool, execute_code_tool]
prompt = """
Task: The attached Excel file contains the sales of menu items for a local fast-food chain. What were the total sales that the chain made from food (not including drinks)? Express your answer in USD with two decimal places.

Incorporate the results from the context, there might be intermediary steps that are already done.

Context:
[
  {
    "provided_by": "user",
    "content": "file provided with the main task",
    "link": "/Users/maksim.rostov/pdev/freelectron/free-generalist/evaluation/gaia/2023/validation/7bd855d8-463d-4ed5-93ca-5fe35145f733.xlsx",
    "metadata": {
      "note": "the file is already in the list of available resources"
    }
  }
]
"""

res = code.llm.predict_and_call(prompt=prompt, tools=tools)

In [ ]:
print(res.response)

In [ ]:
print("""
"=== Exploratory Data Analysis for 7bd855d8-463d-4ed5-93ca-5fe35145f733.xlsx ===\n\nShape: 9 rows x 7 columns\n\n\n--- Column Information ---\nColumns: ['Location', 'Burgers', 'Hot Dogs', 'Salads', 'Fries', 'Ice Cream', 'Soda']\n\n\n--- Data Types ---\nLocation     object\nBurgers       int64\nHot Dogs      int64\nSalads        int64\nFries         int64\nIce Cream     int64\nSoda          int64\ndtype: object\n\n--- Summary Statistics ---\n         Location      Burgers     Hot Dogs       Salads        Fries  \\\ncount           9     9.000000     9.000000     9.000000     9.000000   \nunique          9          NaN          NaN          NaN          NaN   \ntop     Pinebrook          NaN          NaN          NaN          NaN   \nfreq            1          NaN          NaN          NaN          NaN   \nmean          NaN  1952.333333  2000.333333  2006.000000  2005.555556   \nstd           NaN   136.278208    18.668155    15.483863    12.836580   \nmin           NaN  1594.000000  1969.000000  1982.000000  1989.000000   \n25%           NaN  1967.000000  1999.000000  2001.000000  1994.000000   \n50%           NaN  2007.000000  2008.000000  2012.000000  2005.000000   \n75%           NaN  2015.000000  2010.000000  2018.000000  2015.000000   \nmax           NaN  2019.000000  2022.000000  2022.000000  2023.000000   \n\n          Ice Cream         Soda  \ncount      9.000000     9.000000  \nunique          NaN          NaN  \ntop             NaN          NaN  \nfreq            NaN          NaN  \nmean    2003.111111  2116.444444  \nstd       18.884150   319.201782  \nmin     1969.000000  1980.000000  \n25%     1998.000000  2009.000000  \n50%     2005.000000  2016.000000  \n75%     2017.000000  2019.000000  \nmax     2021.000000  2967.000000  \n\n--- Missing Values ---\nNo missing values found", sources=[ToolOutput(blocks=[TextBlock(block_type='text', text="=== Exploratory Data Analysis for 7bd855d8-463d-4ed5-93ca-5fe35145f733.xlsx ===\n\nShape: 9 rows x 7 columns\n\n\n--- Column Information ---\nColumns: ['Location', 'Burgers', 'Hot Dogs', 'Salads', 'Fries', 'Ice Cream', 'Soda']\n\n\n--- Data Types ---\nLocation     object\nBurgers       int64\nHot Dogs      int64\nSalads        int64\nFries         int64\nIce Cream     int64\nSoda          int64\ndtype: object\n\n--- Summary Statistics ---\n         Location      Burgers     Hot Dogs       Salads        Fries  \\\ncount           9     9.000000     9.000000     9.000000     9.000000   \nunique          9          NaN          NaN          NaN          NaN   \ntop     Pinebrook          NaN          NaN          NaN          NaN   \nfreq            1          NaN          NaN          NaN          NaN   \nmean          NaN  1952.333333  2000.333333  2006.000000  2005.555556   \nstd           NaN   136.278208    18.668155    15.483863    12.836580   \nmin           NaN  1594.000000  1969.000000  1982.000000  1989.000000   \n25%           NaN  1967.000000  1999.000000  2001.000000  1994.000000   \n50%           NaN  2007.000000  2008.000000  2012.000000  2005.000000   \n75%           NaN  2015.000000  2010.000000  2018.000000  2015.000000   \nmax           NaN  2019.000000  2022.000000  2022.000000  2023.000000   \n\n          Ice Cream         Soda  \ncount      9.000000     9.000000  \nunique          NaN          NaN  \ntop             NaN          NaN  \nfreq            NaN          NaN  \nmean    2003.111111  2116.444444  \nstd       18.884150   319.201782  \nmin     1969.000000  1980.000000  \n25%     1998.000000  2009.000000  \n50%     2005.000000  2016.000000  \n75%     2017.000000  2019.000000  \nmax     2021.000000  2967.000000  \n\n--- Missing Values ---\nNo missing values found")], tool_name='do_table_eda', raw_input={'args': ('/Users/maksim.rostov/pdev/freelectron/free-generalist/evaluation/gaia/2023/validation/7bd855d8-463d-4ed5-93ca-5fe35145f733.xlsx',), 'kwargs': {}}, raw_output="=== Exploratory Data Analysis for 7bd855d8-463d-4ed5-93ca-5fe35145f733.xlsx ===\n\nShape: 9 rows x 7 columns\n\n\n--- Column Information ---\nColumns: ['Location', 'Burgers', 'Hot Dogs', 'Salads', 'Fries', 'Ice Cream', 'Soda']\n\n\n--- Data Types ---\nLocation     object\nBurgers       int64\nHot Dogs      int64\nSalads        int64\nFries         int64\nIce Cream     int64\nSoda          int64\ndtype: object\n\n--- Summary Statistics ---\n         Location      Burgers     Hot Dogs       Salads        Fries  \\\ncount           9     9.000000     9.000000     9.000000     9.000000   \nunique          9          NaN          NaN          NaN          NaN   \ntop     Pinebrook          NaN          NaN          NaN          NaN   \nfreq            1          NaN          NaN          NaN          NaN   \nmean          NaN  1952.333333  2000.333333  2006.000000  2005.555556   \nstd           NaN   136.278208    18.668155    15.483863    12.836580   \nmin           NaN  1594.000000  1969.000000  1982.000000  1989.000000   \n25%           NaN  1967.000000  1999.000000  2001.000000  1994.000000   \n50%           NaN  2007.000000  2008.000000  2012.000000  2005.000000   \n75%           NaN  2015.000000  2010.000000  2018.000000  2015.000000   \nmax           NaN  2019.000000  2022.000000  2022.000000  2023.000000   \n\n          Ice Cream         Soda  \ncount      9.000000     9.000000  \nunique          NaN          NaN  \ntop             NaN          NaN  \nfreq            NaN          NaN  \nmean    2003.111111  2116.444444  \nstd       18.884150   319.201782  \nmin     1969.000000  1980.000000  \n25%     1998.000000  2009.000000  \n50%     2005.000000  2016.000000  \n75%     2017.000000  2019.000000  \nmax     2021.000000  2967.000000  \n\n--- Missing Values ---\nNo missing values found"
""")